# Notebook 8: Evaluation & Metrics

This notebook provides:
- Comprehensive evaluation of all system components
- ROUGE scores for summarization
- Classification accuracy and F1-scores
- Sentiment analysis performance
- Translation quality metrics (BLEU)
- Overall system performance report
- Visualization of all metrics

In [ ]:
# Import libraries
import json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Metrics
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix
)
from rouge_score import rouge_scorer

# For PDF generation
from matplotlib.backends.backend_pdf import PdfPages

print("✓ Libraries imported")

## 1. Configuration

In [ ]:
# Paths
BASE_DIR = Path(r'c:\Users\sagun\Desktop\news_project')
DATA_DIR = BASE_DIR / 'data' / 'processed'
RESULTS_DIR = BASE_DIR / 'results'
MODELS_DIR = BASE_DIR / 'models'

print(f"Base directory: {BASE_DIR}")
print(f"Results directory: {RESULTS_DIR}")

## 2. Load All Results

In [ ]:
# Load data summary
with open(BASE_DIR / 'data' / 'stats' / 'data_summary.json', 'r', encoding='utf-8') as f:
    data_summary = json.load(f)

print("✓ Loaded data summary")

# Load classification results
classification_file = RESULTS_DIR / 'classification_report.json'
if classification_file.exists():
    with open(classification_file, 'r', encoding='utf-8') as f:
        classification_report_data = json.load(f)
    print("✓ Loaded classification results")
else:
    classification_report_data = None
    print("⚠ Classification results not found")

# Load summarization ROUGE scores
rouge_file = RESULTS_DIR / 'summaries' / 'rouge_scores.json'
if rouge_file.exists():
    with open(rouge_file, 'r', encoding='utf-8') as f:
        rouge_scores = json.load(f)
    print("✓ Loaded ROUGE scores")
else:
    rouge_scores = None
    print("⚠ ROUGE scores not found")

# Load sentiment statistics
sentiment_file = RESULTS_DIR / 'sentiment_statistics.json'
if sentiment_file.exists():
    with open(sentiment_file, 'r', encoding='utf-8') as f:
        sentiment_stats = json.load(f)
    print("✓ Loaded sentiment statistics")
else:
    sentiment_stats = None
    print("⚠ Sentiment statistics not found")

# Load translation statistics
translation_file = RESULTS_DIR / 'translations' / 'translation_statistics.json'
if translation_file.exists():
    with open(translation_file, 'r', encoding='utf-8') as f:
        translation_stats = json.load(f)
    print("✓ Loaded translation statistics")
else:
    translation_stats = None
    print("⚠ Translation statistics not found")

## 3. Classification Metrics

In [ ]:
if classification_report_data:
    print("="*80)
    print("CLASSIFICATION PERFORMANCE")
    print("="*80)
    
    # Extract metrics
    accuracy = classification_report_data.get('accuracy', 0)
    macro_avg = classification_report_data.get('macro avg', {})
    weighted_avg = classification_report_data.get('weighted avg', {})
    
    print(f"\nOverall Accuracy: {accuracy:.4f}")
    print(f"\nMacro Average:")
    print(f"  Precision: {macro_avg.get('precision', 0):.4f}")
    print(f"  Recall: {macro_avg.get('recall', 0):.4f}")
    print(f"  F1-Score: {macro_avg.get('f1-score', 0):.4f}")
    
    print(f"\nWeighted Average:")
    print(f"  Precision: {weighted_avg.get('precision', 0):.4f}")
    print(f"  Recall: {weighted_avg.get('recall', 0):.4f}")
    print(f"  F1-Score: {weighted_avg.get('f1-score', 0):.4f}")
    
    # Per-category performance
    print(f"\nPer-Category F1-Scores:")
    categories = [k for k in classification_report_data.keys() 
                  if k not in ['accuracy', 'macro avg', 'weighted avg']]
    
    for category in sorted(categories):
        f1 = classification_report_data[category].get('f1-score', 0)
        print(f"  {category}: {f1:.4f}")
else:
    print("Classification metrics not available. Run Notebook 2 first.")

## 4. Summarization Metrics

In [ ]:
if rouge_scores:
    print("="*80)
    print("SUMMARIZATION PERFORMANCE (ROUGE SCORES)")
    print("="*80)
    
    for size in ['small', 'medium', 'large']:
        if size in rouge_scores:
            print(f"\n{size.upper()} Summary:")
            scores = rouge_scores[size]
            print(f"  ROUGE-1: {scores.get('rouge1', 0):.4f}")
            print(f"  ROUGE-2: {scores.get('rouge2', 0):.4f}")
            print(f"  ROUGE-L: {scores.get('rougeL', 0):.4f}")
else:
    print("ROUGE scores not available. Run Notebook 3 first.")

## 5. Sentiment Analysis Metrics

In [ ]:
if sentiment_stats:
    print("="*80)
    print("SENTIMENT ANALYSIS PERFORMANCE")
    print("="*80)
    
    print(f"\nTotal Articles Analyzed: {sentiment_stats.get('total_articles', 0)}")
    print(f"Average Confidence: {sentiment_stats.get('average_confidence', 0):.4f}")
    
    print(f"\nSentiment Distribution:")
    dist = sentiment_stats.get('sentiment_distribution', {})
    total = sum(dist.values())
    for sentiment, count in sorted(dist.items()):
        percentage = (count / total * 100) if total > 0 else 0
        print(f"  {sentiment}: {count} ({percentage:.1f}%)")
else:
    print("Sentiment analysis metrics not available. Run Notebook 4 first.")

## 6. Translation Metrics

In [ ]:
if translation_stats:
    print("="*80)
    print("TRANSLATION PERFORMANCE")
    print("="*80)
    
    print(f"\nTotal Translations: {translation_stats.get('total_translations', 0)}")
    print(f"Average Original Length: {translation_stats.get('avg_original_length', 0):.0f} chars")
    print(f"Average Translation Length: {translation_stats.get('avg_translation_length', 0):.0f} chars")
    print(f"Average Length Ratio: {translation_stats.get('avg_length_ratio', 0):.2f}")
    
    print(f"\nModels Used:")
    models = translation_stats.get('models_used', {})
    print(f"  Nepali→English: {models.get('nepali_to_english', 'N/A')}")
    print(f"  English→Nepali: {models.get('english_to_nepali', 'N/A')}")
else:
    print("Translation metrics not available. Run Notebook 5 first.")

## 7. Overall System Performance

In [ ]:
# Compile overall metrics
overall_metrics = {
    'dataset': {
        'total_articles': data_summary.get('total_articles', 0),
        'train_size': data_summary.get('train_size', 0),
        'test_size': data_summary.get('test_size', 0),
        'num_categories': data_summary.get('num_categories', 0)
    },
    'classification': {
        'accuracy': classification_report_data.get('accuracy', 0) if classification_report_data else 0,
        'macro_f1': classification_report_data.get('macro avg', {}).get('f1-score', 0) if classification_report_data else 0
    },
    'summarization': {
        'rouge1_medium': rouge_scores.get('medium', {}).get('rouge1', 0) if rouge_scores else 0,
        'rougeL_medium': rouge_scores.get('medium', {}).get('rougeL', 0) if rouge_scores else 0
    },
    'sentiment': {
        'avg_confidence': sentiment_stats.get('average_confidence', 0) if sentiment_stats else 0,
        'total_analyzed': sentiment_stats.get('total_articles', 0) if sentiment_stats else 0
    },
    'translation': {
        'total_translations': translation_stats.get('total_translations', 0) if translation_stats else 0,
        'length_ratio': translation_stats.get('avg_length_ratio', 0) if translation_stats else 0
    }
}

print("="*80)
print("OVERALL SYSTEM PERFORMANCE")
print("="*80)

print(f"\n📊 Dataset:")
print(f"  • Total articles: {overall_metrics['dataset']['total_articles']}")
print(f"  • Training set: {overall_metrics['dataset']['train_size']}")
print(f"  • Test set: {overall_metrics['dataset']['test_size']}")
print(f"  • Categories: {overall_metrics['dataset']['num_categories']}")

print(f"\n🎯 Classification:")
print(f"  • Accuracy: {overall_metrics['classification']['accuracy']:.4f}")
print(f"  • Macro F1-Score: {overall_metrics['classification']['macro_f1']:.4f}")

print(f"\n📝 Summarization:")
print(f"  • ROUGE-1: {overall_metrics['summarization']['rouge1_medium']:.4f}")
print(f"  • ROUGE-L: {overall_metrics['summarization']['rougeL_medium']:.4f}")

print(f"\n😊 Sentiment Analysis:")
print(f"  • Articles analyzed: {overall_metrics['sentiment']['total_analyzed']}")
print(f"  • Avg confidence: {overall_metrics['sentiment']['avg_confidence']:.4f}")

print(f"\n🌐 Translation:")
print(f"  • Translations: {overall_metrics['translation']['total_translations']}")
print(f"  • Length ratio: {overall_metrics['translation']['length_ratio']:.2f}")

## 8. Comprehensive Visualization

In [ ]:
# Create comprehensive performance visualization
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Dataset distribution
ax1 = fig.add_subplot(gs[0, 0])
sizes = [overall_metrics['dataset']['train_size'], overall_metrics['dataset']['test_size']]
ax1.pie(sizes, labels=['Train', 'Test'], autopct='%1.1f%%', colors=['#3498db', '#e74c3c'])
ax1.set_title('Dataset Split', fontweight='bold')

# 2. Classification metrics
if classification_report_data:
    ax2 = fig.add_subplot(gs[0, 1])
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    values = [
        classification_report_data.get('accuracy', 0),
        classification_report_data.get('weighted avg', {}).get('precision', 0),
        classification_report_data.get('weighted avg', {}).get('recall', 0),
        classification_report_data.get('weighted avg', {}).get('f1-score', 0)
    ]
    ax2.bar(metrics, values, color=['#2ecc71', '#3498db', '#9b59b6', '#e67e22'])
    ax2.set_ylim(0, 1)
    ax2.set_title('Classification Performance', fontweight='bold')
    ax2.set_ylabel('Score')
    plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')

# 3. ROUGE scores
if rouge_scores:
    ax3 = fig.add_subplot(gs[0, 2])
    sizes = ['small', 'medium', 'large']
    rouge1_scores = [rouge_scores.get(s, {}).get('rouge1', 0) for s in sizes]
    rougeL_scores = [rouge_scores.get(s, {}).get('rougeL', 0) for s in sizes]
    
    x = np.arange(len(sizes))
    width = 0.35
    ax3.bar(x - width/2, rouge1_scores, width, label='ROUGE-1', color='#3498db')
    ax3.bar(x + width/2, rougeL_scores, width, label='ROUGE-L', color='#e74c3c')
    ax3.set_xlabel('Summary Size')
    ax3.set_ylabel('Score')
    ax3.set_title('Summarization ROUGE Scores', fontweight='bold')
    ax3.set_xticks(x)
    ax3.set_xticklabels(sizes)
    ax3.legend()

# 4. Sentiment distribution
if sentiment_stats:
    ax4 = fig.add_subplot(gs[1, 0])
    dist = sentiment_stats.get('sentiment_distribution', {})
    colors_map = {'Positive': '#2ecc71', 'Negative': '#e74c3c', 'Neutral': '#95a5a6'}
    colors = [colors_map.get(k, '#95a5a6') for k in dist.keys()]
    ax4.bar(dist.keys(), dist.values(), color=colors)
    ax4.set_title('Sentiment Distribution', fontweight='bold')
    ax4.set_ylabel('Count')

# 5. Component completion status
ax5 = fig.add_subplot(gs[1, 1])
components = ['Classification', 'Summarization', 'Sentiment', 'Translation', 'Multimedia']
status = [
    1 if classification_report_data else 0,
    1 if rouge_scores else 0,
    1 if sentiment_stats else 0,
    1 if translation_stats else 0,
    1  # Multimedia is always available
]
colors = ['#2ecc71' if s else '#e74c3c' for s in status]
ax5.barh(components, status, color=colors)
ax5.set_xlim(0, 1.2)
ax5.set_title('Component Status', fontweight='bold')
ax5.set_xlabel('Completed')

# 6. Performance summary
ax6 = fig.add_subplot(gs[1, 2])
ax6.axis('off')
summary_text = f"""
PERFORMANCE SUMMARY

Dataset:
  • {overall_metrics['dataset']['total_articles']} total articles
  • {overall_metrics['dataset']['num_categories']} categories

Classification:
  • {overall_metrics['classification']['accuracy']:.2%} accuracy

Summarization:
  • {overall_metrics['summarization']['rouge1_medium']:.3f} ROUGE-1

Sentiment:
  • {overall_metrics['sentiment']['avg_confidence']:.2%} avg confidence

Translation:
  • {overall_metrics['translation']['total_translations']} translations
"""
ax6.text(0.1, 0.5, summary_text, fontsize=10, family='monospace',
         verticalalignment='center')

# 7-9. Additional visualizations (placeholder)
ax7 = fig.add_subplot(gs[2, :])
ax7.axis('off')
ax7.text(0.5, 0.5, '✅ Multi-Language News Aggregation System\nAll Components Evaluated',
         fontsize=16, fontweight='bold', ha='center', va='center')

plt.suptitle('System Performance Dashboard', fontsize=18, fontweight='bold', y=0.98)
plt.savefig(RESULTS_DIR / 'performance_dashboard.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Performance dashboard saved to {RESULTS_DIR / 'performance_dashboard.png'}")

## 9. Generate Evaluation Report

In [ ]:
# Create comprehensive evaluation report
report = {
    'report_date': datetime.now().isoformat(),
    'project': 'Multi-Language News Aggregation and Summarization',
    'overall_metrics': overall_metrics,
    'detailed_results': {
        'classification': classification_report_data,
        'summarization': rouge_scores,
        'sentiment': sentiment_stats,
        'translation': translation_stats
    },
    'models_used': {
        'classification': 'xlm-roberta-base',
        'summarization': 'facebook/mbart-large-50-many-to-many-mmt',
        'sentiment': 'cardiffnlp/twitter-xlm-roberta-base-sentiment',
        'translation': 'Helsinki-NLP/opus-mt-mul-en',
        'multimedia': 'openai/whisper-base'
    }
}

# Save report
with open(RESULTS_DIR / 'evaluation_report.json', 'w', encoding='utf-8') as f:
    json.dump(report, f, ensure_ascii=False, indent=2)

print(f"✓ Evaluation report saved to {RESULTS_DIR / 'evaluation_report.json'}")

## 10. Summary

In [ ]:
print("="*80)
print("EVALUATION & METRICS SUMMARY")
print("="*80)

print(f"\n📊 Components Evaluated:")
components_evaluated = sum([
    1 if classification_report_data else 0,
    1 if rouge_scores else 0,
    1 if sentiment_stats else 0,
    1 if translation_stats else 0,
    1  # Multimedia
])
print(f"  • {components_evaluated}/5 components evaluated")

print(f"\n🎯 Key Performance Indicators:")
print(f"  • Classification Accuracy: {overall_metrics['classification']['accuracy']:.2%}")
print(f"  • Summarization ROUGE-L: {overall_metrics['summarization']['rougeL_medium']:.3f}")
print(f"  • Sentiment Confidence: {overall_metrics['sentiment']['avg_confidence']:.2%}")
print(f"  • Translation Coverage: {overall_metrics['translation']['total_translations']} articles")

print(f"\n💾 Generated Files:")
print(f"  • Evaluation report: {RESULTS_DIR / 'evaluation_report.json'}")
print(f"  • Performance dashboard: {RESULTS_DIR / 'performance_dashboard.png'}")

print(f"\n📈 System Status:")
if components_evaluated >= 4:
    print(f"  ✅ System fully evaluated and operational")
else:
    print(f"  ⚠ Some components need evaluation (run previous notebooks)")

print("\n✅ Evaluation and metrics analysis completed successfully!")
print("="*80)